# 🎯 Gweizy Model Training Notebook

Train ML models for gas price prediction on Google Colab.

**Steps:**
1. Upload your `gas_data.db` file from `backend/gas_data.db`
2. Run all cells (Runtime → Run all)
3. Download trained models zip
4. Extract and copy to `backend/models/saved_models/`
5. Commit and push to deploy!

## 1️⃣ Install Dependencies

In [ ]:
%pip install pandas numpy scikit-learn joblib lightgbm sqlalchemy python-dateutil tqdm -q
print("✅ Dependencies installed!")

## 2️⃣ Upload Database

Upload your `gas_data.db` file from `backend/gas_data.db`

In [ ]:
from google.colab import files
import os

# Check if database already exists
if not os.path.exists('gas_data.db'):
    print("📁 Please upload your gas_data.db file:")
    uploaded = files.upload()
    print(f"✅ Uploaded: {list(uploaded.keys())}")
else:
    print("✅ Database already exists!")

## 3️⃣ Load and Inspect Data

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime, timedelta
import time
import warnings
warnings.filterwarnings('ignore')

start_time = time.time()
def log(msg):
    elapsed = time.time() - start_time
    print(f"[{elapsed:6.1f}s] {msg}")

# Connect to database
engine = create_engine('sqlite:///gas_data.db')

# Load data (note: column is 'current_gas' not 'gas_price')
query = """
SELECT timestamp, current_gas, block_number, base_fee, priority_fee
FROM gas_prices
ORDER BY timestamp DESC
"""

df = pd.read_sql(query, engine)
# Rename to gas_price for consistency with feature engineering code
df = df.rename(columns={'current_gas': 'gas_price'})

log(f"📊 Loaded {len(df):,} records")
log(f"📅 Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
log(f"⛽ Gas price range: {df['gas_price'].min():.6f} to {df['gas_price'].max():.6f} gwei")
df.head()

## 4️⃣ Feature Engineering

In [ ]:
log("🔧 Starting feature engineering...")

# Sort by timestamp
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)

# Sample if too large (use all data on Colab - we have resources!)
MAX_RECORDS = 100000  # Can handle more on Colab
if len(df) > MAX_RECORDS:
    log(f"⚠️ Sampling {MAX_RECORDS:,} from {len(df):,} records")
    recent = df.tail(MAX_RECORDS // 5)
    older = df.head(len(df) - MAX_RECORDS // 5).sample(MAX_RECORDS - len(recent), random_state=42)
    df = pd.concat([older, recent]).sort_values('timestamp').reset_index(drop=True)
    log(f"✅ Using {len(df):,} records")

# Outlier capping
Q1, Q3 = df['gas_price'].quantile([0.25, 0.75])
IQR = Q3 - Q1
lower, upper = Q1 - 3*IQR, Q3 + 3*IQR
outliers = ((df['gas_price'] < lower) | (df['gas_price'] > upper)).sum()
log(f"⚠️ Capping {outliers:,} outliers ({outliers/len(df)*100:.1f}%)")
df['gas_price'] = df['gas_price'].clip(lower, upper)

# Time features
log("   Adding time features...")
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

# Lag features
log("   Adding lag features...")
for lag in [1, 2, 3, 6, 12, 24]:
    df[f'gas_lag_{lag}'] = df['gas_price'].shift(lag)

# Rolling statistics
log("   Adding rolling statistics...")
for window in [6, 12, 24, 48]:
    df[f'gas_ma_{window}'] = df['gas_price'].rolling(window).mean()
    df[f'gas_std_{window}'] = df['gas_price'].rolling(window).std()
    df[f'gas_min_{window}'] = df['gas_price'].rolling(window).min()
    df[f'gas_max_{window}'] = df['gas_price'].rolling(window).max()

# Price change features
log("   Adding price change features...")
df['gas_pct_change_1'] = df['gas_price'].pct_change(1) * 100
df['gas_pct_change_6'] = df['gas_price'].pct_change(6) * 100
df['gas_pct_change_12'] = df['gas_price'].pct_change(12) * 100
df['gas_pct_change_24'] = df['gas_price'].pct_change(24) * 100

# Volatility
df['volatility_6h'] = df['gas_price'].rolling(6).std() / df['gas_price'].rolling(6).mean()
df['volatility_24h'] = df['gas_price'].rolling(24).std() / df['gas_price'].rolling(24).mean()

# Momentum
df['momentum_6'] = df['gas_price'] - df['gas_price'].shift(6)
df['momentum_12'] = df['gas_price'] - df['gas_price'].shift(12)
df['momentum_24'] = df['gas_price'] - df['gas_price'].shift(24)

# EMA
df['ema_6'] = df['gas_price'].ewm(span=6).mean()
df['ema_12'] = df['gas_price'].ewm(span=12).mean()
df['ema_24'] = df['gas_price'].ewm(span=24).mean()

# Drop NaN rows
initial_len = len(df)
df = df.dropna()
log(f"✅ Features created: {len(df):,} samples, {len(df.columns)} features")
print(f"\n📊 Feature columns: {list(df.columns)}")

## 5️⃣ Create Targets & Prepare Features

In [ ]:
log("🎯 Creating prediction targets...")

# Estimate steps per hour from data
# Ensure timestamp is datetime
if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
    df['timestamp'] = pd.to_datetime(df['timestamp'])

time_diffs = df['timestamp'].diff().dropna()
if len(time_diffs) == 0:
    log("⚠️ No time differences found, using default: 1 step per hour")
    steps_per_hour = 1
else:
    median_interval = time_diffs.median()
    if pd.isna(median_interval):
        log("⚠️ Could not calculate median interval, using default: 1 step per hour")
        steps_per_hour = 1
    else:
        median_interval_min = median_interval.total_seconds() / 60  # minutes
        if median_interval_min <= 0 or not np.isfinite(median_interval_min):
            log("⚠️ Invalid interval, using default: 1 step per hour")
            steps_per_hour = 1
        else:
            steps_per_hour = max(1, int(60 / median_interval_min))
            log(f"   Detected {steps_per_hour} steps per hour (interval: {median_interval_min:.1f} min)")

# Future price targets
horizons = {
    '1h': steps_per_hour * 1,
    '4h': steps_per_hour * 4,
    '24h': steps_per_hour * 24
}

targets = {}
for name, steps in horizons.items():
    future_price = df['gas_price'].shift(-steps)
    pct_change = ((future_price - df['gas_price']) / (df['gas_price'] + 1e-8)) * 100
    targets[name] = {
        'pct_change': pct_change,
        'original': future_price,
        'current': df['gas_price'].copy()
    }
    valid = (~pct_change.isna()).sum()
    log(f"   {name}: {valid:,} valid targets, steps={steps}")
    
    # Diagnostic info
    if valid == 0:
        log(f"      ⚠️ WARNING: No valid targets for {name}!")
        log(f"      Total rows: {len(df):,}, Shift steps: {steps}")
        log(f"      Future price NaN: {(future_price.isna()).sum():,}")
        log(f"      Current price NaN: {(df['gas_price'].isna()).sum():,}")

# Select feature columns
exclude_cols = ['timestamp', 'block_number']
feature_cols = [c for c in df.columns if c not in exclude_cols]
X = df[feature_cols].copy()
log(f"📊 Feature matrix: {X.shape[0]:,} samples, {X.shape[1]} features")

## 6️⃣ Train Models

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

# Create output directory
os.makedirs('trained_models', exist_ok=True)

results = {}

for horizon in ['1h', '4h', '24h']:
    log(f"\n{'='*60}")
    log(f"🌲 Training model for {horizon} horizon")
    log(f"{'='*60}")
    
    y_pct = targets[horizon]['pct_change']
    y_orig = targets[horizon]['original']
    current_prices = targets[horizon]['current']
    
    # Remove NaN
    valid_idx = ~(X.isna().any(axis=1) | y_pct.isna() | y_orig.isna())
    X_clean = X[valid_idx]
    y_pct_clean = y_pct[valid_idx]
    y_orig_clean = y_orig[valid_idx]
    current_clean = current_prices[valid_idx]
    
    log(f"   Valid samples: {len(X_clean):,}")
    
    # Check if we have enough data
    MIN_SAMPLES = 100  # Minimum samples needed for training
    if len(X_clean) < MIN_SAMPLES:
        log(f"   ⚠️ SKIPPING: Only {len(X_clean):,} valid samples (need at least {MIN_SAMPLES})")
        log(f"   💡 This might be because:")
        log(f"      - Not enough historical data for {horizon} horizon")
        log(f"      - Too many NaN values after feature engineering")
        log(f"      - Data gaps in timestamp sequence")
        continue
    
    # Train/test split (80/20, temporal)
    split_idx = int(len(X_clean) * 0.8)
    X_train, X_test = X_clean.iloc[:split_idx], X_clean.iloc[split_idx:]
    y_train, y_test = y_pct_clean.iloc[:split_idx], y_pct_clean.iloc[split_idx:]
    y_orig_test = y_orig_clean.iloc[split_idx:]
    current_test = current_clean.iloc[split_idx:]
    
    log(f"   Train: {len(X_train):,}, Test: {len(X_test):,}")
    
    # Additional check after split
    if len(X_train) == 0:
        log(f"   ⚠️ SKIPPING: No training samples after split")
        continue
    
    # Scale features
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Train RandomForest
    log(f"   Training RandomForest (this may take 1-2 min)...")
    model = RandomForestRegressor(
        n_estimators=200,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train_scaled, y_train)
    
    # Predict
    y_pred_pct = model.predict(X_test_scaled)
    y_pred_orig = current_test.values * (1 + y_pred_pct / 100)
    
    # Metrics
    mae = mean_absolute_error(y_orig_test, y_pred_orig)
    rmse = np.sqrt(mean_squared_error(y_orig_test, y_pred_orig))
    r2 = r2_score(y_orig_test, y_pred_orig)
    mape = np.mean(np.abs((y_orig_test - y_pred_orig) / (y_orig_test + 1e-8))) * 100
    
    # Directional accuracy
    y_diff_actual = np.diff(y_orig_test.values)
    y_diff_pred = np.diff(y_pred_orig)
    dir_acc = np.mean(np.sign(y_diff_actual) == np.sign(y_diff_pred))
    
    log(f"   ✅ R²: {r2:.4f}")
    log(f"   ✅ MAE: {mae:.6f} gwei")
    log(f"   ✅ MAPE: {mape:.2f}%")
    log(f"   ✅ Directional Accuracy: {dir_acc*100:.1f}%")
    
    # Feature importance
    importances = model.feature_importances_
    top_features = sorted(zip(feature_cols, importances), key=lambda x: x[1], reverse=True)[:5]
    log(f"   Top features: {[f[0] for f in top_features]}")
    
    # Save model
    model_data = {
        'model': model,
        'model_name': 'RandomForest_PctChange',
        'feature_scaler': scaler,
        'feature_names': feature_cols,
        'predicts_percentage_change': True,
        'uses_log_scale': False,
        'metrics': {'mae': mae, 'rmse': rmse, 'r2': r2, 'mape': mape, 'directional_accuracy': dir_acc},
        'trained_at': datetime.now().isoformat(),
        'training_samples': len(X_train),
        'feature_importances': dict(zip(feature_cols, importances))
    }
    
    model_path = f'trained_models/model_{horizon}.pkl'
    joblib.dump(model_data, model_path)
    log(f"   💾 Saved to {model_path}")
    
    scaler_path = f'trained_models/scaler_{horizon}.pkl'
    joblib.dump(scaler, scaler_path)
    log(f"   💾 Saved scaler to {scaler_path}")
    
    results[horizon] = model_data['metrics']

log(f"\n{'='*60}")
log("🎉 TRAINING COMPLETE!")
log(f"{'='*60}")

## 7️⃣ Summary & Download

In [ ]:
import shutil

print("📊 Model Performance Summary:")
print("="*50)
if len(results) == 0:
    print("\n⚠️ No models were successfully trained!")
    print("   This usually means:")
    print("   - Not enough historical data")
    print("   - Data gaps preventing target creation")
    print("   - Too many NaN values after feature engineering")
    print("\n   💡 Try:")
    print("   - Uploading a database with more historical data")
    print("   - Checking that timestamps are properly formatted")
    print("   - Ensuring gas_price values are valid")
else:
    for horizon, metrics in results.items():
        print(f"\n{horizon}:")
        print(f"  R²: {metrics['r2']:.4f}")
        print(f"  MAE: {metrics['mae']:.6f} gwei")
        print(f"  MAPE: {metrics['mape']:.2f}%")
        print(f"  Directional Accuracy: {metrics['directional_accuracy']*100:.1f}%")

print("\n" + "="*50)
if os.path.exists('trained_models') and len(os.listdir('trained_models')) > 0:
    print("📁 Generated files:")
    for f in os.listdir('trained_models'):
        size = os.path.getsize(f'trained_models/{f}') / 1024 / 1024
        print(f"  • {f} ({size:.1f} MB)")
else:
    print("⚠️ No model files generated")

# Create zip if models exist
if os.path.exists('trained_models') and len(os.listdir('trained_models')) > 0:
    shutil.make_archive('gweizy_models', 'zip', 'trained_models')
    print("\n📦 Created gweizy_models.zip")
    
    # Download
    files.download('gweizy_models.zip')
    print("\n✅ Download started!")
    print("\n📋 Next steps:")
    print("1. Extract gweizy_models.zip")
    print("2. Copy model_*.pkl to backend/models/saved_models/")
    print("3. git add, commit, push")
    print("4. Railway will auto-deploy with new models!")
else:
    print("\n⚠️ Cannot create zip - no models were trained")
    print("   Please check your data and try again")